In [3]:
# %rm -rf phase2_module
# !git clone https://github.com/LgDNet/phase2_module.git
# !cd phase2_module && make copy

In [1]:
cd ..

/home/elicer


/home/elicer/.local/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [3]:
from phase2_module.src.utils.dataset import Data
from phase2_module.src.models.classification.models import Model
from phase2_module.src.base import BasePiepline
# from phase2_module.src.hyper_parameters.params_optimization import xgboost_opt, set_params_optimization_data
from phase2_module.src.utils.set_seed import seed_everything
from phase2_module.src.utils.top_score_instance import check_the_score
from phase2_module.src.encoder.encoders import Encode

from phase2_module.src.utils.manage_pkl_files import PickleManager as pkl_manager
from phase2_module.src import preprocess
seed_everything()  # NOTE: default 42
_model = Model()

In [4]:
import pandas as pd
import numpy as np
import re, pickle
import warnings
from tqdm import tqdm
warnings.filterwarnings(action='ignore')
def set_up(mode= None):
    pd.set_option('display.max_columns', mode)
set_up()

# Preprocessing

In [8]:
basic = preprocess.Basic
country = preprocess.Country
customer = preprocess.Customer
inquiry = preprocess.Inquiry
product_category = preprocess.ProductCategory
convert_ratio = preprocess.Convert_ratio
customer2 = preprocess.Customer2
category = preprocess.Categorization
business = preprocess.Business

# Main

In [9]:
pkls = pkl_manager.map(pkl_manager.loads, pkl_manager.metadata_directory)

runtime_instance = {
    "Train": Data.train,
    "Test": Data.test,
    "Instances":[
        {"instance": basic, "method": ['drop_duplicated'], "params": False},
        {"instance": country, "method": ['apply'], "params": pkls.get("country")},
        {"instance": inquiry, "method": ["apply"], "params": False},
        {"instance": product_category, "method": ["product_categories", 'fill_odds'], "params": {**pkls.get("product_category"), **pkls.get("product_subcategory")}},
        {"instance": customer, "method": ["apply"], "params": pkls.get("customer")},
        {"instance": category, "method": ["cat_customer_idx",'historical_cnt_mean'], "params": False},
        {"instance": convert_ratio, "method": ["lead_owner"], "params": {**pkls.get("lead_owner"), **pkls.get("customer_idx")}},
        #{"instance": basic, "method": ['customer_idx_merge_enterprise'], "params": False},
        {"instance": customer2, "method": ['seniority_level', 'job_function', 'customer_type'], "params": pkls.get("customer2")},
        {"instance": business, "method": ["apply"], "params": pkls.get("business")},
        {"instance": category, "method": ['customer_idx_merge_enterprise'], "params": False},
    ]
}

## preprocessing

In [42]:
df_train = preprocess.runner('Train',**runtime_instance)
df_test = preprocess.runner('Test',**runtime_instance)

In [44]:
df_train['business_area'].value_counts()

business_area
corporate / office                26177
education                          8304
residential (home)                 7378
retail                             3853
hotel & accommodation              2859
factory                            2494
special purpose                    2029
hospital & health care             1082
government department               635
public facility                     503
transportation                      291
power plant / renewable energy      108
Space                                50
Name: count, dtype: int64

In [7]:
cond = df_train['is_converted'] == True
true_num = df_train[cond]['lead_desc_length'].mean()
cond1 = df_train['lead_desc_length'] > true_num
df_train.loc[cond1,'less_timeline'] += 1
cond2 = df_test['lead_desc_length'] > true_num
df_test.loc[cond2,'less_timeline'] += 1

## encoding

In [45]:
encode = Encode()

In [46]:
exisiting_columns = [
                     'business_area','business_subarea','business_unit',
                        'enterprise',
#                         'response_corporate',#"customer_country", # country
#                         'customer_type','customer_job','customer_position', #customer
#                         'inquiry_type','expected_timeline', # inquiry
#                         'product_subcategory','product_category','product_modelname' # product_category
                       ] 
preprocess_columns = [
                     'country','continent', # country
                     'customer_type2','job_function','seniority_level', # customer
                     'new_expected_timeline', 'new_inquiry_type',# inquiry
                     'category_2','category_3','category_1',
                     'new_customer_type1','new_customer_type2' # product_category
#                      'test2'
                     ]

label_encode_columns = exisiting_columns + preprocess_columns

In [47]:
# 결측치 채우기
df_train = preprocess.fillna(df_train)
df_test = preprocess.fillna(df_test)

In [11]:
# # one-hot encoding
# onehot_encode_columns = ['category_1']
# df_train = encode.onehot_encoder(df_train,onehot_encode_columns)
# df_test = encode.onehot_encoder(df_test,onehot_encode_columns)

In [48]:
# label encoding
encode.set_up() # 전역 변수 초기화
df_train = encode.label_encoder(df_train,label_encode_columns)
df_test = encode.label_encoder(df_test,label_encode_columns)

# Modeling

## drop column
모델링 전이 column을 drop하면서 다루기가 쉽다고 판단하였음

In [49]:
df_train.columns

Index(['bant_submit', 'customer_country', 'business_unit',
       'com_reg_ver_win_rate', 'customer_idx', 'customer_type',
       'historical_existing_cnt', 'id_strategic_ver', 'it_strategic_ver',
       'idit_strategic_ver', 'customer_job', 'lead_desc_length',
       'inquiry_type', 'product_category', 'product_subcategory',
       'product_modelname', 'customer_country.1', 'customer_position',
       'response_corporate', 'expected_timeline', 'ver_cus', 'ver_pro',
       'ver_win_rate_x', 'ver_win_ratio_per_bu', 'business_area',
       'business_subarea', 'lead_owner', 'is_converted', 'country',
       'continent', 'new_inquiry_type', 'new_expected_timeline',
       'less_timeline', 'customer_interest', 'category_1', 'category_2',
       'category_3', 'customer_type2', 'job_function', 'seniority_level',
       'new_customer_idx', 'historical_existing_mean',
       'lead_owner_converted_ratio', 'enterprise', 'new_customer_type1',
       'new_customer_type2'],
      dtype='object')

In [50]:
drop_col = [
            'id_strategic_ver','it_strategic_ver','idit_strategic_ver','ver_pro','ver_cus',
            'customer_country.1',
            "customer_country",#'response_corporate' # country
            'customer_type','customer_job','customer_position', # customer
            'inquiry_type', 'expected_timeline', #inquiry
            'product_subcategory','product_category','product_modelname', # product_category
#             'cate_is_nan', # product_category
    
#             'com_reg_ver_win_rate','idit_strategic_ver','ver_cus',
#             'ver_pro','ver_win_rate_x','ver_win_ratio_per_bu',
            'customer_idx',
            'lead_owner','historical_existing_mean','response_corporate',#'new_customer_idx'
            'customer_type2', 'business_subarea',
#             'customer_idx_converted_ratio', # ratio
            #'lead_owner', 'lead_owner_converted_ratio', 'lead_owner_customer_idx_mean', # lead_owner
            ]
df_train.drop(drop_col,axis = 1, inplace = True)
df_test.drop(drop_col,axis = 1, inplace = True)

## Scaler

In [51]:
import numpy as np
# log 변환
df_train['lead_desc_length'] = np.log1p(df_train['lead_desc_length'])
df_test['lead_desc_length'] = np.log1p(df_test['lead_desc_length'])

In [52]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
df_train[['lead_owner_converted_ratio']] = scaler.fit_transform(df_train[['lead_owner_converted_ratio']])
df_test[['lead_owner_converted_ratio']] = scaler.transform(df_test[['lead_owner_converted_ratio']])

## type

In [53]:
df_train['new_customer_idx'] = df_train['new_customer_idx'].astype(int)
df_test['new_customer_idx'] = df_test['new_customer_idx'].astype(int)

## train

#### param

In [54]:
import math
from six.moves import xrange

class LoglossObjective(object):
    def calc_ders_range(self, approxes, targets, weights):
        assert len(approxes) == len(targets)
        if weights is not None:
            assert len(weights) == len(approxes)
        
        result = []
        for index in range(len(targets)):
            e = np.exp(approxes[index])
            p = e / (1 + e)
            der1 = targets[index] - p
            der2 = -p * (1 - p)
            

            if (targets[index] == 0.0) & (p<0.5):
                #( True Negative )
                der1 = (p)*(targets[index] - p)
                der2 = (1-2*p)*(-p * (1 - p))
                
            elif (targets[index] == 0.0) & (p>=0.5):
                # 타겟값이 노말 유저인데,fraud 라고 예측한 경우 ( False Positive )
                der1 = 3*(p)*(targets[index] - p)
                der2 = 3*(1-2*p)*(-p * (1 - p))

            elif (targets[index] > 0.0) :
                # 타겟 값이 fraud 이라면 p값이 높을수록 (정답에 가까울수록) 페널티가 적도록 세팅. 추가로 3배 페널티
                der1 = 3*(1-p)*(targets[index] - p)
                der2 = 3*(2*p-1)*(-p * (1 - p))
                
            if weights is not None:
                der1 *= weights[index]
                der2 *= weights[index]

            result.append((der1, der2))
        return result

In [55]:
cat_params = {'iterations':5000, 'depth':12, 'thread_count':3, 'learning_rate':1e-3, 'loss_function':LoglossObjective(), 'eval_metric':'PRAUC', 'early_stopping_rounds':400, 'verbose':1000}
decision_params = {'criterion': 'log_loss', 'max_depth': 20, 'min_samples_split': 7, 'min_samples_leaf': 2, 'random_state': 42}
xgb_params = {'n_estimators': 2393, 'max_depth': 16, 'min_child_weight': 2, 'learning_rate': 0.06783336462585174, 'subsample': 0.9511145375386896, 'gamma': 0.16846211770550093, 'colsample_bytree': 0.6662896237544925, 'colsample_bylevel': 0.8781042717330898, 'colsample_bynode': 0.741271661588135, 'random_state':42}
lgb_params = {'num_iterations' : 5000, 'objective' : 'binary', 'is_unbalance':True, 'max_depth': -1, 'early_stopping_rounds' : 500, 'verbose':0, 'seed':42}
random_params = {'max_depth': 19, 'max_leaf_nodes': 749, 'n_estimators': 349,'random_state':42}
gbm_params = {'n_estimators':1000, 'learning_rate' : 1e-3, 'max_depth' : 16, 'random_state' : 42,'verbose':2, }

### train_

In [56]:
X = df_train.drop(["is_converted"],axis = 1)
Y = df_train['is_converted']

#Y = np.where(Y == False, 0, 1) # catboost

In [57]:
X = X.apply(pd.to_numeric)

In [52]:
# # oversampling
# from imblearn.over_sampling import SMOTE, ADASYN
# adasyn = SMOTE(sampling_strategy=0.2,random_state=42)
# X, Y = adasyn.fit_resample(X, Y)

In [41]:
def score_print(result_dict):
    for name, score_list in result_dict.items():
        print(f'{name} score : {np.mean(score_list):.4f} / STD: (+/- {np.std(score_list):.4f})')

In [70]:
import copy
from collections import defaultdict
from sklearn.model_selection import StratifiedKFold
stratkfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
def kfold(fit_model, param):
    train_result, valid_result = defaultdict(list), defaultdict(list) # result 리스트
    models = [] # 모델을 저장할 리스트
    # k-fold
    for idx,(train_idx, test_idx) in tqdm(enumerate(stratkfold.split(X,Y))):
        base_model = copy.deepcopy(fit_model)
        
        x_train, x_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = Y[train_idx], Y[test_idx]
        # 모델 선언
        model = base_model
        model.set_params(**param)
        # 모델 훈련
        if model.__class__.__name__ != 'LGBMClassifier':
            model.fit(x_train, y_train)            
        else:
            model.fit(x_train, y_train,eval_set=[(x_test, y_test)],eval_metric = LoglossObjective())
            
        models.append(model) # 훈련된 모델을 리스트에 추가
        train_pred, valid_pred = models[idx].predict(x_train), models[idx].predict(x_test)
        # store result
        for name, score in check_the_score(train_pred, y_train).items(): # train
            train_result[name].append(score)
        for name, score in check_the_score(valid_pred, y_test).items():  # valid
            valid_result[name].append(score)
    # print score
    print(f'----[{model.__class__.__name__} K-Fold Train Score]-----')
    score_print(train_result)
    print(f'----[{model.__class__.__name__} K-Fold Validation Score]-----')
    score_print(valid_result)
    print('-'*50)
    return models

5it [00:01,  3.83it/s]

----[K-Fold Train Score]-----
f1 score : 0.9363 / STD: (+/- 0.0018)
precision score : 0.9109 / STD: (+/- 0.0038)
recall score : 0.9632 / STD: (+/- 0.0027)
roc_auc_score score : 0.9776 / STD: (+/- 0.0013)
----[K-Fold Validation Score]-----
f1 score : 0.7819 / STD: (+/- 0.0113)
precision score : 0.7682 / STD: (+/- 0.0109)
recall score : 0.7962 / STD: (+/- 0.0132)
roc_auc_score score : 0.8876 / STD: (+/- 0.0070)


In [ ]:
model_params = {
#                 'cat': (_model.catboost, cat_params),
#                 'gbm': (_model.gradient_boosting, gbm_params),
                'decision': (_model.decision_tree,decision_params),
#                 'xgb': (_model.xgboost, xgb_params),
                'lgb': (_model.lightgbm,lgb_params),
#                 'random': (_model.random_forest, random_params),
                }

In [ ]:
models = defaultdict(list)
for model_name, (model, param) in model_params.items():
    models[model_name] = kfold(model,param)

In [32]:
# threshold valid
threshold = 0.5
for i, model_name in enumerate(models.keys()):
    valid_result = defaultdict(list) # result 리스트
    for idx, (_, test_idx) in tqdm(enumerate(stratkfold.split(X,Y))):
        x_test, y_test = X.iloc[test_idx], Y[test_idx]
        y_valid_pred = models[model_name][idx].predict_proba(x_test)[:,0]
        y_valid_pred = np.where(y_valid_pred >= threshold, False, True)

        for name, score in check_the_score(y_valid_pred, y_test).items():  # valid
            valid_result[name].append(score)
    # print score
    print(f'{models[model_name][idx].__class__.__name__}----[K-Fold Validation Score]-----')
    score_print(valid_result)
    

5it [00:00,  7.02it/s]

----[K-Fold Validation Score]-----
f1 score : 0.8345 / STD: (+/- 0.0045)
precision score : 0.8249 / STD: (+/- 0.0107)
recall score : 0.8446 / STD: (+/- 0.0095)
roc_auc_score score : 0.9144 / STD: (+/- 0.0045)


In [59]:
# feature importance
feature_importances = model.feature_importances_
feature_names = x_train.columns
feature_importance_dict = dict(zip(feature_names, feature_importances))
sorted_importance_dict = {k: v for k, v in sorted(feature_importance_dict.items(), key=lambda item: item[1], reverse=True)}
for feature_name, importance in sorted_importance_dict.items():
    print(f"{feature_name}: {importance}")

enterprise: 0.22575248777866364
lead_owner_converted_ratio: 0.16142994165420532
new_customer_idx: 0.07247424870729446
ver_win_rate_x: 0.04723936691880226
new_customer_type1: 0.04195321351289749
com_reg_ver_win_rate: 0.038468796759843826
ver_win_ratio_per_bu: 0.03626057505607605
new_customer_type2: 0.03625210374593735
business_unit: 0.03338652104139328
country: 0.031055809929966927
category_2: 0.028496170416474342
historical_existing_cnt: 0.026527803391218185
continent: 0.020842717960476875
category_1: 0.02007449045777321
new_expected_timeline: 0.019922547042369843
lead_desc_length: 0.019889118149876595
category_3: 0.01942816935479641
bant_submit: 0.019287889823317528
new_inquiry_type: 0.018824564293026924
customer_interest: 0.018071362748742104
business_area: 0.017808780074119568
job_function: 0.016775159165263176
less_timeline: 0.015035528689622879
seniority_level: 0.014742611907422543


#### inference

In [65]:
df_test = df_test.apply(pd.to_numeric)
test_drop = df_test.drop(['id','is_converted'],axis = 1)
df_test.dtypes

id                              int64
bant_submit                   float64
business_unit                   int64
com_reg_ver_win_rate          float64
historical_existing_cnt       float64
lead_desc_length              float64
ver_win_rate_x                float64
ver_win_ratio_per_bu          float64
business_area                   int64
is_converted                     bool
country                         int64
continent                       int64
new_inquiry_type                int64
new_expected_timeline           int64
less_timeline                 float64
customer_interest               int64
category_1                      int64
category_2                      int64
category_3                      int64
job_function                    int64
seniority_level                 int64
new_customer_idx                int64
lead_owner_converted_ratio    float64
enterprise                      int64
new_customer_type1              int64
new_customer_type2              int64
dtype: objec

In [66]:
# voting threshold
predicts = defaultdict(list)
threshold = 0.9
for model_name in models.keys():
    for model in models[model_name]:
        pred_proba = model.predict_proba(test_drop)[:,0]
        pred = np.where(pred_proba >= threshold, False, True)
        predicts[model_name].append(pred)

In [67]:
for name in predicts.keys():
    print(name)
    for i in predicts[name]:
        print(sum(i))
    print('-'*10)

1174
1180
1217
1195
1205


In [68]:
final_predicts={}
for idx, name in enumerate(predicts.keys()):
    num_threshold = 1
    globals()[f'pred{idx+1}'] = np.where( np.sum(predicts[name],axis = 0) >= num_threshold, 1, 0)
    globals()[f'pred{idx+1}'] = np.where(globals()[f'pred{idx+1}'] == 0, False, True)
    print(name)
    print(sum(globals()[f'pred{idx+1}']))
    final_predicts[name] = globals()[f'pred{idx+1}']

1402

## ensemble

In [69]:
date='0223'
for name, pred in final_predicts.items():
    df_test['is_converted'] = pred
    df_test.to_csv(f"{date}_{name}.csv",index = False)

In [5]:

# decision_tree: num_threshold: 1 + threshold: 0.9
# xgb: num_threshold: 1, threshold = 0.9  -> 1402
# lgb: num_threshold: 1 + threshold: 0.8
# ramdom_forest: num_threshold: 1 + threshold: 0.9
# cat: num_threshold: 1 + threshold: 0.7
# gbm: num_threshold: 1 + threshold: 0.9
# 1682 1446 1609 1770 1753 1688

In [21]:
# load csv
model_list = ['decision_tree','xgb','lgb','random_forest','cat','gbm','svm']
for idx, model_name in enumerate(model_list):
    df = pd.read_csv(f"{date}_{model_name}.csv")
    globals()[f'pred{idx+1}'] = np.array(df['is_converted'])

In [22]:
for idx in range(len(model_list)):
    print(sum(globals()[f'pred{idx+1}']), end = ' ')

1682 1446 1609 1770 1753 1688 1338 

In [23]:
ensemble_ = np.array([globals()['pred'+ str(idx+1)] for idx in range(len(model_list))])

In [27]:
ensemble_threshold = 4
pred = np.where( np.sum(ensemble_,axis = 0) >= ensemble_threshold, 1, 0)
pred = np.where(pred == 0, False, True)
sum(pred)

1585

## submission

In [28]:
df_test['is_converted'] = pred
df_test.to_csv("submission.csv",index = False)

In [29]:
sum(pred)

1585